In [5]:
from pok_env_gym_RLLib import PokemonEnv
import yaml

In [6]:
from ray.tune.registry import register_env

with open("env_config.yaml", 'r') as file:
    env_config = yaml.safe_load(file)

def env_creator(env_config):
    return PokemonEnv(config=env_config, render_mode='human')  # return an env instance

register_env("my_env", env_creator)

In [7]:
env = PokemonEnv(config=env_config)
print(env.observation_space)
print(env._get_obs())
print(env._get_info())

Box(0, 255, (100, 100), uint8)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
{'current_hp': 0.0, 'levels': 0, 'badges': 0, 'pokedex_count': 0}


In [14]:
from ray.rllib.algorithms.dqn.dqn import DQNConfig, DQN
config = DQNConfig()

replay_config = {
        "type": "MultiAgentPrioritizedReplayBuffer",
        "capacity": 60000,
        "prioritized_replay_alpha": 0.5,
        "prioritized_replay_beta": 0.5,
        "prioritized_replay_eps": 3e-6,
    }

config = config.training(replay_buffer_config=replay_config,
                         target_network_update_freq=4,
                         n_step=1,
                         num_atoms=1,
                         double_q=False,
                         dueling=False)
config = config.resources(num_gpus=0)
config = config.rollouts(num_rollout_workers=2)
config = config.environment("my_env", env_config=env_config)
algo = DQN(config=config)
algo.train()


2024-03-07 11:00:51,442	WARNING util.py:62 -- Install gputil for GPU system monitoring.


{'custom_metrics': {},
 'episode_media': {},
 'info': {'learner': {},
  'num_env_steps_sampled': 1000,
  'num_env_steps_trained': 0,
  'num_agent_steps_sampled': 1000,
  'num_agent_steps_trained': 0},
 'sampler_results': {'episode_reward_max': nan,
  'episode_reward_min': nan,
  'episode_reward_mean': nan,
  'episode_len_mean': nan,
  'episode_media': {},
  'episodes_this_iter': 0,
  'policy_reward_min': {},
  'policy_reward_max': {},
  'policy_reward_mean': {},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [], 'episode_lengths': []},
  'sampler_perf': {},
  'num_faulty_episodes': 0,
  'connector_metrics': {}},
 'episode_reward_max': nan,
 'episode_reward_min': nan,
 'episode_reward_mean': nan,
 'episode_len_mean': nan,
 'episodes_this_iter': 0,
 'policy_reward_min': {},
 'policy_reward_max': {},
 'policy_reward_mean': {},
 'hist_stats': {'episode_reward': [], 'episode_lengths': []},
 'sampler_perf': {},
 'num_faulty_episodes': 0,
 'connector_metrics': {},
 'num_healthy_wo